# Exploring Early SARS-CoV2 Mutations
We are working in the directory *SARS-CoV-2*.

## Install Programs

We need two AWK scripts

In [10]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/fasta2tbl'

--2024-02-10 10:58:47--  https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/fasta2tbl
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/fasta2tbl [following]
--2024-02-10 10:58:47--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/fasta2tbl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299 [text/plain]
Saving to: ‘fasta2tbl.1’

fasta2tbl.1         100%[===================>]     299  --.-KB/s    in 0s      

2024-02-10 10:58:47 (586 KB/s) - ‘fasta2tbl.1’ saved [299/299]



In [9]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/compare-cov2.awk'

--2024-02-10 10:44:06--  https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/compare-cov2.awk
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/compare-cov2.awk [following]
--2024-02-10 10:44:06--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/compare-cov2.awk
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2904 (2.8K) [text/plain]
Saving to: ‘compare-cov2.awk.1’

compare-cov2.awk.1  100%[===================>]   2.84K  --.-KB/s    in 0.003s  

2024-02-10 10:44:07 (855 KB/s) - ‘compare-cov2.awk.1’ saved [2904/2904]



## Download Virus Sequences

Download reference genome:

In [1]:
efetch -db nuccore -id NC_045512 -format fasta > wuhan-1.fasta

We create a copy in tab-delimited format:

In [13]:
./fasta2tbl wuhan-1.fasta > wuhan-1.tab

Download from [NCBI](https://www.ncbi.nlm.nih.gov/sars-cov-2/) viruses from Europe, from human hosts, without ambigious characters, complete nucleotide sequences, and a sequence length of exactly 29,903 nt. They are downloaded via the web browser as *sequences.fasta*. Move them into your current working directory.

Rename the file to *cov2-len-29903.fasta*

In [2]:
mv sequences.fasta cov2-len-29903.fasta

In [3]:
grep -c ">" cov2-len-29903.fasta

3371


## Convert FASTA to TAB and Edit Header

In [11]:
./fasta2tbl sequences.fasta | sed 's/|/\t/g' > sequences.tab

In [12]:
wc -l sequences.tab

3371 sequences.tab
